In [ ]:
import requests

url = 'https://api.bilibili.com/pgc/view/web/season?season_id=33378'
res = requests.get(url, headers={'User-Agent':'Chrome/87.0.4280.88'})

with open('tmp_playlist.json', 'w', encoding='utf8') as f:
    f.write(res.text)

In [ ]:
import json

with open('tmp_playlist.json', 'r', encoding='utf8') as f:
    data = json.load(f)

assert data['code'] == 0
assert data['message'] == 'success'

videos = []

for ep in data['result']['episodes']:
    x = {}
    videos.append(x)

    x['playId'] = 'ep' + str(ep['id'])
    x['tvid'] = int(ep['title'])
    x['title'] = ep['long_title']

len(videos)
videos[3]

with open('tvlist.json', 'w', encoding='utf8') as f:
    json.dump(videos, f, ensure_ascii=False, indent=4)

In [ ]:
# extract wiki by 海外版id

# zh.wikipedia.org/wiki/名偵探柯南動畫集數列表

from bs4 import BeautifulSoup
from typing import List
from itertools import islice
import re

s = BeautifulSoup(open('wiki-zh.html', 'r', encoding='utf8'))

def normalize(s:str):
    s = re.sub(r'\[\d+?\]', '', s)
    s = re.sub(r'（[^（]+?特别版）','',s)
    s = re.sub(r'[\s\r\n“”《》！·!]','',s)
    return s

def extract(table, cols:List[int], skip_rows=2) -> List:
    info_list = []
    for tr in islice(table.select('tr'), skip_rows, None):
        tds = tr.select('td')
        info = []
        for i in cols:
            info.append(tds[i].text)
        info[1] = normalize(info[1])
        
        # extract tvid
        idcol = tds[1].text

        # 有括号(顿号分隔)；没括号(可以转换为int)
        m = re.findall(r'\(([^\(\)]+?)\)', idcol)
        if len(m) > 0:
            eps = m[0].split('、')
            eps = [int(p) for p in eps]
        else:
            try: 
                eps = [int(idcol)]
            except:
                eps = []
                idcol:str
                ok = idcol.startswith('魔术快斗第') or idcol == '番外特别篇'
                if not ok:
                    raise

        for ep in eps:
            info_list.append([ep]+info)

    return info_list

table_count = 13

col_list = [3] * table_count
col_list[0] = 4
col_list = [[0, p] for p in col_list]

tables = s.select('.wikitable')
len(tables)

records = []
for i in range(table_count):
    x = extract(tables[i], col_list[i])
    records += x

with open('wiki-by-tvid.json', 'w', encoding='utf8') as f:
    json.dump(records, f, indent=4, ensure_ascii=False)

In [ ]:
# compare wiki tvid and bilibili -- 用眼睛看
result_file = 'tv-with-time.json'

import json

wiki2 = json.load(open('wiki-by-tvid.json', 'r', encoding='utf8'))
bili = json.load(open('tvlist.json', 'r', encoding='utf8'))

# wiki to dict
wiki2_d = {p[0]:p for p in wiki2}

for tv in bili:
    tvid = tv['tvid']
    title = tv['title']

    wiki_title = wiki2_d[tvid][2]
    if title != wiki_title:
        if normalize(title) != wiki_title:
            print(title, '--', wiki_title)
    
    # join the data
    pat = r'^(\d+)年(\d+)月(\d+)日$'
    ymd = list(re.findall(pat, wiki2_d[tvid][1])[0])
    tv['pub'] = '/'.join(ymd)

json.dump(bili, open(result_file,'w',encoding='utf8'), ensure_ascii=False, indent=4)